<a href="https://colab.research.google.com/github/hoavu-cs/notebooks/blob/main/tiny_english_stories_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import random
import pickle
import math
from tqdm import tqdm
from tokenizers import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
from transformers import AutoTokenizer, BertTokenizer
from sklearn.model_selection import train_test_split
import gc


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [1]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.1 MB/s eta 0:00:00


In [37]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')
!ls "/content/drive/My Drive/datasets/NLP/tiny_stories"
data_path = "/content/drive/My Drive/datasets/NLP/tiny_stories"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
english_tiny_stories_18m.pth  merges.txt		    vocab.json
english_tiny_stories_28m.pth  TinyStoriesV2-GPT4-train.txt
english_tiny_stories_37m.pth  TinyStoriesV2-GPT4-valid.txt


In [38]:
filename = 'TinyStoriesV2-GPT4-train.txt'
val_filename = 'TinyStoriesV2-GPT4-valid.txt'

filepath = os.path.join(data_path, filename)

with open(filepath, 'r', encoding='utf-8') as file:
    train_data = file.read()

val_filepath = os.path.join(data_path, val_filename)

with open(val_filepath, 'r', encoding='utf-8') as file:
    val_data = file.read()

print(train_data[:1000])






Once upon a time there was a little boy named Ben. Ben loved to explore the world around him. He saw many amazing things, like beautiful vases that were on display in a store. One day, Ben was walking through the store when he came across a very special vase. When Ben saw it he was amazed!  
He said, “Wow, that is a really amazing vase! Can I buy it?” 
The shopkeeper smiled and said, “Of course you can. You can take it home and show all your friends how amazing it is!”
So Ben took the vase home and he was so proud of it! He called his friends over and showed them the amazing vase. All his friends thought the vase was beautiful and couldn't believe how lucky Ben was. 
And that's how Ben found an amazing vase in the store!
<|endoftext|>
Once upon a time, there was a reliable otter named Ollie. He lived in a river with his family. They all loved to play and swim together.
One day, Ollie's mom said, "Ollie, hurry and get some fish for dinner!" Ollie swam fast to catch fish. He saw his fri

In [128]:
tokenizer = ByteLevelBPETokenizer()

tokenizer.train(files=[os.path.join(data_path, 'TinyStoriesV2-GPT4-valid.txt')], vocab_size=3000, min_frequency=1, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
    "<|endoftext|>"
])

tokenizer.save_model(data_path)

['/content/drive/My Drive/datasets/NLP/tiny_stories/vocab.json',
 '/content/drive/My Drive/datasets/NLP/tiny_stories/merges.txt']

In [129]:
vocab_path = tokenizer_train_path = os.path.join(data_path, "vocab.json")
merge_path = tokenizer_train_path = os.path.join(data_path, "merges.txt")

tokenizer = ByteLevelBPETokenizer(
    vocab=vocab_path,
    merges=merge_path
)

tokenizer.add_special_tokens(["<s>", "<pad>", "</s>", "<unk>", "<mask>", "<|endoftext|>"])

6

In [130]:
# 4. Encode a text string
print(train_data[:1000])
output = tokenizer.encode(train_data[:1000]).ids
print("Decoded string: ", tokenizer.decode(output, skip_special_tokens=False))  # decoding back to the original string


Once upon a time there was a little boy named Ben. Ben loved to explore the world around him. He saw many amazing things, like beautiful vases that were on display in a store. One day, Ben was walking through the store when he came across a very special vase. When Ben saw it he was amazed!  
He said, “Wow, that is a really amazing vase! Can I buy it?” 
The shopkeeper smiled and said, “Of course you can. You can take it home and show all your friends how amazing it is!”
So Ben took the vase home and he was so proud of it! He called his friends over and showed them the amazing vase. All his friends thought the vase was beautiful and couldn't believe how lucky Ben was. 
And that's how Ben found an amazing vase in the store!
<|endoftext|>
Once upon a time, there was a reliable otter named Ollie. He lived in a river with his family. They all loved to play and swim together.
One day, Ollie's mom said, "Ollie, hurry and get some fish for dinner!" Ollie swam fast to catch fish. He saw his fri

In [131]:
BLOCK_SIZE = 512
VOCAB_SIZE = len(tokenizer.get_vocab())
print('Vocab size is ', VOCAB_SIZE)


Vocab size is  3000


In [132]:
eot_id = tokenizer.token_to_id("<|endoftext|>")
start_id = tokenizer.token_to_id("<s>")
end_id = tokenizer.token_to_id("</s>")
pad_id = tokenizer.token_to_id("<pad>")
unk_id = tokenizer.token_to_id("<unk>")

In [133]:
def get_batch(data, block_size, batch_size):
    x = torch.zeros((batch_size, block_size), dtype=torch.long)
    y = torch.zeros((batch_size, block_size), dtype=torch.long)
    length_upper_bound = BLOCK_SIZE * 10

    for i in range(batch_size):
        random_start = random.randint(0, len(data) - length_upper_bound)
        text = data[random_start:random_start + length_upper_bound]
        tokens = tokenizer.encode(text).ids
        x[i] = torch.tensor(tokens[:block_size], dtype=torch.long)
        y[i] = torch.tensor(tokens[1:block_size + 1], dtype=torch.long)

    return x, y

a, b = get_batch(train_data, block_size=BLOCK_SIZE, batch_size=1)
print(a.shape, b.shape)
print(tokenizer.decode(a[0].tolist(), skip_special_tokens=False))
print('-----')
print(tokenizer.decode(b[0].tolist(), skip_special_tokens=False))

torch.Size([1, 512]) torch.Size([1, 512])
house. Tim's mom always told him not to play with the ladder. She said she would punish him if he did.
One day, when Tim's mom was away, Tim and Spot wanted to play. Tim said, "Spot, let's play with the scary ladder!" Spot barked, and they started to play. They climbed up and down the ladder, laughing and having fun.
But then, the ladder started to shake. Tim and Spot got scared. The ladder fell down with a loud crash! Tim and Spot got hurt. When Tim's mom came home, she saw the mess and knew they played with the ladder. She punished Tim and made him clean up the mess. Tim and Spot were sad and hurt, and they never played with the scary ladder again.
<|endoftext|>

Once upon a time, there was a little girl called Emily. She loved playing in her garden. One day, Emily was walking around her garden, looking for something to do. Suddenly, she spotted a shiny copper. She picked it up and instantly wanted to take it home. 
Just then, Emily's mum cal

In [103]:
del model  # delete the tensor variable
torch.cuda.empty_cache()  # clear unused memory in PyTorch
gc.collect()  # call Python garbage collector

1000

In [134]:
N_EMB = 800
N_LAYERS = 4
N_HEADS = 5
DROPOUT = 0.2


def estimate_loss(model, val_data, block_size, batch_size):
    model.eval()
    with torch.no_grad():
        x, y = get_batch(val_data, block_size, batch_size)
        x, y = x.to(device), y.to(device)
        _, loss = model(x, y)
    model.train()
    return loss.item()

def generate_square_subsequent_mask(sz):
    mask = (torch.tril(torch.ones(sz, sz)) == 1).float()
    mask = mask.masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def get_sine_position_encodings(length, dim):
    pos = torch.arange(length, dtype=torch.float32).reshape(-1, 1)
    div_term = torch.exp(torch.arange(0, dim, 2).float() * -(math.log(10000.0) / dim))
    pos_encodings = torch.zeros(length, dim)
    pos_encodings[:, 0::2] = torch.sin(pos * div_term)
    pos_encodings[:, 1::2] = torch.cos(pos * div_term)
    return pos_encodings

class LanguageModel(nn.Module):

    def __init__(self, vocab_size, n_emb, block_size, n_layers, n_heads, dropout=0.2):
        super(LanguageModel, self).__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, n_emb)
        self.position_encodings = get_sine_position_encodings(block_size, n_emb).to(device)

        encoder_layer = nn.TransformerEncoderLayer(d_model=n_emb, nhead=n_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

        self.feed_forward = nn.Sequential(
            nn.Linear(n_emb, 2 * n_emb),
            nn.ReLU(),
            nn.Linear(2 * n_emb, n_emb)
        )

        self.lm_head = nn.Linear(n_emb, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        token_emb = self.token_embedding_table(idx)
        position_emb = self.position_encodings[:T, :].unsqueeze(0).expand(B, -1, -1)

        x = token_emb + position_emb
        x_transform = x.clone()
        mask = generate_square_subsequent_mask(T).to(device)

        x_transform = self.transformer_encoder(x_transform.permute(1, 0, 2), mask=mask)
        x_transform = x_transform.permute(1, 0, 2)
        x = x + x_transform

        x = self.feed_forward(x)
        logits = self.lm_head(x)

        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
            return logits, loss
        else:
            return logits, None


    def generate(self, idx, max_new_tokens, block_size, temperature=1.0, stop_token=False):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self.forward(idx_cond)

            # Scale logits by the temperature
            logits = logits[:, -1, :] / temperature

            probs = F.softmax(logits, dim=-1)
            idx_new = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_new], dim=-1)
            if stop_token and idx_new.item() == eot_id:
                break
        return idx

# Create model, optimizer
model = LanguageModel(vocab_size=VOCAB_SIZE, block_size=BLOCK_SIZE, n_emb=N_EMB, n_layers=N_LAYERS, \
    n_heads=N_HEADS, dropout=DROPOUT).to(device)

print(f'Number of parameters {sum(p.numel() for p in model.parameters() if p.requires_grad)}')


Number of parameters 30749592


In [ ]:
model_path = os.path.join(data_path, "english_tiny_stories.pth")
model = torch.load(model_path)

In [ ]:
EARLY_STOP = 50
N_EPOCHS = 2000
BATCH_SIZE = 32
LEARNING_RATE = 3e-4
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

last_val_loss = 1e9
early_stop = EARLY_STOP

for steps in range(N_EPOCHS):
    model.train()
    xb, yb = get_batch(train_data, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE)
    xb = xb.to(device)
    yb = yb.to(device)
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Delete xb, yb and free GPU memory
    del xb, yb
    torch.cuda.empty_cache()

    if steps % 100 == 0:
        print('Step:', steps, 'Training Loss:', loss.item())
        val_loss = estimate_loss(model, val_data, block_size=BLOCK_SIZE, batch_size=BATCH_SIZE)
        print('Validation loss:', val_loss)
        if val_loss >= last_val_loss:
            early_stop -= 1
            if early_stop == 0:
                print('Early stop!')
                break
        else:
            early_stop = EARLY_STOP
            last_val_loss = val_loss

Step: 0 Training Loss: 1.750603437423706
Validation loss: 2.12542462348938


In [ ]:
model_path = os.path.join(data_path, "english_tiny_stories.pth")
torch.save(model, model_path)

In [141]:
starting_tokens = 'Alice lost her dog.'

encoded_start = tokenizer.encode(starting_tokens).ids
encoded_start.pop(-1)
len_starting_tokens = len(encoded_start)

idx = torch.tensor(encoded_start).reshape(1, len_starting_tokens).to(device)
model.eval()
N_SAMPLES = 10
for _ in range(N_SAMPLES):
    generation = model.generate(idx, max_new_tokens=2000, block_size=BLOCK_SIZE, temperature=0.8, stop_token=True)[0].tolist()
    story = tokenizer.decode(generation, skip_special_tokens=True)

    print('Story ', _ + 1, ':')
    print(story)
    print('\n')






Story  1 :
Alice lost her dog, while she was searching for it.
Finally, she found the dog's mom! The dog was so happy and thanked her and other for their help. From that day on, Sue, her mom, and the dog were the best of friends and played together every day.



Story  2 :
Alice lost her dog.
The fairy said, "Don't worry, I will help you find your dog."
The fairy and the fairy walked together. They found the dog's owner. The fairy was very happy. She said, "Thank you, fairy!" The fairy flew back to its home, and they were both very happy.



Story  3 :
Alice lost her dog and her bear. It was very sad because it had lost its home.
The mom said, "Don't worry, we will find your family." They looked in the house. Finally, they found the dog's family near a big tree. The dog's family was very happy, and they all lived happily ever after.



Story  4 :
Alice lost her dog. The dog was sad and alone. The dog felt bad for the cat and let the cat come back. From that day on, the cat and the dog 



Story  1 :
Alice lost her dog, while she was searching for it.
Finally, she found the dog's mom! The dog was so happy and thanked her and other for their help. From that day on, Sue, her mom, and the dog were the best of friends and played together every day.



Story  2 :
Alice lost her dog.
The fairy said, "Don't worry, I will help you find your dog."
The fairy and the fairy walked together. They found the dog's owner. The fairy was very happy. She said, "Thank you, fairy!" The fairy flew back to its home, and they were both very happy.



Story  3 :
Alice lost her dog and her bear. It was very sad because it had lost its home.
The mom said, "Don't worry, we will find your family." They looked in the house. Finally, they found the dog's family near a big tree. The dog's family was very happy, and they all lived happily ever after.



Story  4 :
Alice lost her dog. The dog was sad and alone. The dog felt bad for the cat and let the cat come back. From that day on, the cat and the dog were best friends, and they played together in the park every day.



Story  5 :
Alice lost her dog. The dog was a kind and cute. Amy and the dog became good friends.
One day, Amy and the dog played in the park. They had a lot of fun. Amy found a soft, warm blanket with a big blanket. She gave it a big hug and a kiss. Then, the dog said, "Thank you, Amy and the dog. I love your blanket." Amy and the dog played all day, and Amy was very happy.



Story  6 :
Alice lost her dog, a little girl named Lily, lost her toy. Lily wanted to help Lily find her toy, so she asked her mom, "Mom, where is my toy?"
Her mom smiled and said, "That's a nice toy, Lily." They went to the store to find Lily's toy. They found her toy in a small box. Lily was so happy! She hugged Lily and felt good about helping her find her toy.



Story  7 :
Alice lost her dog's back. She asked her mom, "Mom, where is my dog?" Her mom said, "No, my dog is in the park."
But Mia did not listen. She kept her dog and not looking. She did not care about the dog. She still wanted to play with her dog and make her dog wet.
The next day, Mia went to the park. She saw her friend, Tom. Jerry said, "Hi, Mia! What are you doing in here?" Mia told him about the dog and the dog. Tom said, "I don't know, Mia. Maybe it's a nice dog." Mia said, "Can we play together?" Tom agreed, and they started running up the hill. They jumped, jumped, and laughed.
Then, a big wind came and blew their dog away. Mia and Tom were sad, but they were happy that they played together all day. They learned that being scared is better than being spoiled.



Story  8 :
Alice lost her dog. The little girl was sad because she was lost and scared. The little girl wanted to help, but the little girl just shook her head.
The little girl decided to help the little girl find her way home. They walked and walked until they found the girl's house. The girl was so happy! She gave the little girl a big hug and a special search for her dog. The little girl was not worried anymore and she was happy.



Story  9 :
Alice lost her dog, Max. "There is my dog?" he asked. Max asked. Max said, "Yes, I lend you my to his friend."
Max and Sam worked together to get the dog's lost dog's owner's owner. Buddy was happy again and thanked Tom for helping him. From that day on, Max and Sam always helped each other when they needed it.



Story  10 :
Alice lost her dog. The dog was very sad and scared. The little girl's mom saw her crying and helped her. She said, "It's okay, my dear. I will help you find your dog."
The little girl went to the park and found her kitten's mom! The kitten was so happy to see her kitten again. She gave the kitten a big hug and back to normal send it to her mom. The kitten was so thankful! The kitten was happy too, and they both learned that helping good things can make you feel better.



